# Class for Benchmar Step

We have developed a python class that implements the complete benchmark step procedure for **QPE** for a $R_z^n(\vec{\theta})$ operator for a fixed number of: *qubits*, *auxiliar number of qubits* and a *type of angle selection*: the **QPE_RZ** class from package *qpe_rz*.

This notebook shows how this class works

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
# myQLM qpus
from qat.qpus import PyLinalg, CLinalg
qpu_c = CLinalg()
qpu_p = PyLinalg()

In [ ]:
import sys
sys.path.append('../../QPE')
from qpe_rz import QPE_RZ

We need to initialize the class. This should be done with a python dictionary that configures the step benchmark execution. Mandatory keywords for the dictionary are:

* **number_of_qbits**: the number of qubits for apply the Kronecker products of $R_z$
* **auxiliar_qbits_number**: auxiliar number of qubits for performing **QPE**
* **angles**: the angles for the $R_z^n(\vec{\theta})$ operator. It can be:
    * float number: the same angle to all $R_z$
    * list with angles: Each angle will be apply to a $R_z$. Number of elements MUST be equal to number of qubits.
    * string: Two different strings can be provided:
        * *random* : random angles will be provide to each $R_z$
        * *exact* : In this case random angles will be provided to each $R_z$ but the final result it will have a precision realted with the number of auxiliar number of qbuits for the **QPE**


In [ ]:
n_qbits = 5
# Fix the precision of the QPE
aux_qbits = 6
# angles
angles = [np.pi / 2.0 for i in range(n_qbits)]
# Dictionary for configuring the class
qpe_dict = {
    'number_of_qbits' : n_qbits,
    'auxiliar_qbits_number' : aux_qbits,
    'angles' : angles,
    #'angles' : 'random',    
    'qpu' : qpu_c,
    'shots' : 0
}

In [ ]:
qpe_rz_b = QPE_RZ(**qpe_dict)
qpe_rz_b.exe()

When the computation is performed (*exe* method) 2 different metrics are computed:

* Kolmogorov-Smirnov distance (**ks** attribute): it is a metric for comparing probability distributions. If it is near zero then the 2 distribution are very similar.
* Fidelity: (**fidelity** attribute): assumes that the 2 obtained eigenvalues distribution (the theorical and the quantum one) are vectors and computes the cosine of the angle between them. For **fidelity** near one then the vectors are more equal. 

Meanwhile the **Kolmogorov-Smirnov** is the good metric for any configuration of **QPE** of the $R_z^{\otimes n}(\vec{\theta})$ operator. The **fidelity** gives a very good metric when the configuration is such that the eigenvalues can be obtained exactly with the **QPE**.

Make some test for understanding how this metrics works in different configurations.

In [ ]:
print(qpe_rz_b.ks, qpe_rz_b.fidelity)

The class provide acces to other attributes that can be useful like:


* *theorical_eigv_dist*: pandas DataFrame with the theorical eigenvalue distribution
* *quantum_eigv_dist*: pandas DataFrame with the QPE eigenvalue distribution

In [ ]:
qpe_rz_b.theorical_eigv_dist

In [ ]:
qpe_rz_b.quantum_eigv_dist

### General QPE configuration

Here we are going to executes the **QPE** of the $R_z^{\otimes n}(\vec{\theta})$ operator for a random selection  of the angles. In this case the eigenvalues of the operator can not be obtained exactly using  the QPE (we would need inifinite number of qubits for have the correct precision for obtained exactly the eigenvalues). In this case the **fidelity** metric does not work properly, even in the exact simulation execution, but the **Kolmogorov-Smirnov** will work.

For this case we set the variable angles to *random* for selecting random angles for each $R_z$

In [ ]:
#Random Angles
n_qbits = 5
# Fix the precision of the QPE
aux_qbits = 10
# angles
# Dictionary for configuring the class
qpe_dict = {
    'number_of_qbits' : n_qbits,
    'auxiliar_qbits_number' : aux_qbits,
    #'angles' : angles,
    'angles' : 'random',    
    'qpu' : qpu_c,
    #'shots' : 0
}

In [ ]:
qpe_rz_b = QPE_RZ(**qpe_dict)
qpe_rz_b.exe()

In [ ]:
print(qpe_rz_b.ks, qpe_rz_b.fidelity)

As can be seen the **Kolmogorov-Smirnov** will give us a value near of 0, but the **fidelity** is far away from zero. We can plot the distributions for comparing the results

In [ ]:
plt.plot(qpe_rz_b.theorical_eigv_dist['lambda'], qpe_rz_b.theorical_eigv_dist['Probability'], '-')
plt.plot(qpe_rz_b.quantum_eigv_dist['lambda'], qpe_rz_b.quantum_eigv_dist['Probability'], '-o')
plt.xlabel('$\lambda$')
plt.ylabel('Probability')
plt.legend(['Theorical', 'QPE'])

The cumulative distributions give us a more exact view of the similaritary of the distributions

In [ ]:
%matplotlib inline

In [ ]:
plt.plot(qpe_rz_b.theorical_eigv_dist['lambda'], qpe_rz_b.theorical_eigv_dist['Probability'].cumsum(), 'o')
plt.plot(qpe_rz_b.quantum_eigv_dist['lambda'], qpe_rz_b.quantum_eigv_dist['Probability'].cumsum(), '-')
plt.xlabel('$\lambda$')
plt.ylabel('Probability')
plt.legend(['Theorical', 'QPE'])
plt.show()

### Particular configuration

We can fix the angles of the $R_z^{\otimes n}(\vec{\theta})$ operator in such a way that the **QPE** can computes exactly the eigenvalues. In this case the **fidelity** works in a very accurate way. This particular configuration is very useful for assesing the capabilities of a quantum computer for executing **QPE** without the noise that have the general case.

For an exact computation we have to provide to the different angles of the $R_z$'s angles that should be multiple of the minimum angular precision of the **QPE** that will be given by: 

$$\delta \theta = \frac{4 \pi}{2^{m}}$$

where $m$ is the number of auxiliar qbuits of the **QPE**.

Providing the *exact* to the keyword **angles** the class compute the angles of the different $R_z$'s by using:

$$\theta_{i+1} = \theta_i +  a * \delta \theta$$ 

with $\theta_0 = \frac{\pi}{2}$ and $a$ random variable that can be $\{-1,1\}$

In [ ]:
n_qbits = 7
# Fix the precision of the QPE
aux_qbits = 7
# angles. The eigenvalues will be sum of the minimum preccision of the QPE
#angles = [4 * np.pi / 2 ** aux_qbits for i in range(n_qbits)]
angles = 'exact'
# Dictionary for configuring the class
qpe_dict = {
    'number_of_qbits' : n_qbits,
    'auxiliar_qbits_number' : aux_qbits,
    'angles' : angles,
    #'angles' : 'random',    
    'qpu' : qpu_c,
    #'shots' : 0
}

In [ ]:
qpe_rz_b.angles

In [ ]:
qpe_rz_b = QPE_RZ(**qpe_dict)
qpe_rz_b.exe()

In [ ]:
print(qpe_rz_b.ks, qpe_rz_b.fidelity)

As can be seen the **fidelity** now is near to 1. This is because the **QPE**, now, allows to compute exactly the eigenvalues of the operator

In [ ]:
%matplotlib notebook
plt.plot(qpe_rz_b.theorical_eigv_dist['lambda'], qpe_rz_b.theorical_eigv_dist['Probability'], '-')
plt.plot(qpe_rz_b.quantum_eigv_dist['lambda'], qpe_rz_b.quantum_eigv_dist['Probability'], '-o')
plt.xlabel('$\lambda$')
plt.ylabel('Probability')
plt.legend(['Theorical', 'QPE'])

In [ ]:
%matplotlib inline
plt.bar(
    qpe_rz_b.theorical_eigv_dist['lambda'], 
    qpe_rz_b.theorical_eigv_dist['Probability'],
    width = 1 / 2**7
)
plt.xlim(0,1)
plt.ylim(0,0.12)
plt.ylabel(r'$P^{th}_{\lambda,m}\left(\frac{j}{2^m}\right)$')
plt.xlabel(r'$\frac{j}{2^m}$')